<a href="https://colab.research.google.com/github/AlexandreXavierMartins/Sim/blob/main/simpy_aula_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy

Sistema Aberto

In [ ]:
import simpy
import numpy as np

# Definição dos parâmetros
tempo_chegada = 30  # Média de chegada dos carros (segundos)
tempo_atendimento = 20  # Média de atendimento (segundos)
tempo_folga = 5 * 60  # Tempo de folga do atendente (5 minutos)
tempo_total = 1 * 3600  # Tempo total da simulação (2 horas)
fila = []

def atender_carro(carro_id):
  global fila
  fila.append(carro_id)
  print('Carros na fila:', fila)
  with atendente.request() as req:
    yield req
    yield env.timeout(np.random.exponential(tempo_atendimento))
    print(f"Carro {carro_id} atendido em {env.now:.2f}s")
    fila.pop(0)
    print('Carros na fila:', fila)


# Processo de chegada dos carros
def chegada_carros(env):
    carro_id = 0
    while True:
        carro_id += 1
        print(f"Carro {carro_id} chegou em {env.now:.2f}s")
        #Errado
        with atendente.request() as req:
          yield req
          yield env.timeout(np.random.exponential(tempo_atendimento))
          print(f"Carro {carro_id} atendido em {env.now:.2f}s")
          if len(fila) > 0:
            fila.pop(0)
          print('Carros na fila:', fila)
        #Certo
        #env.process(atender_carro(carro_id))
        yield env.timeout(np.random.exponential(tempo_chegada))
# Inicialização da simulação
env = simpy.Environment()
atendente = simpy.Resource(env, capacity=1)
env.process(chegada_carros(env))
env.run(until=tempo_total)

Exercício 1: Crie o processo de atendimento do carro no posto de pedágio. O tempo de atendimento é EXPO(20)
a) Colete a ocupação do atendente
b) O tempo médio de fila
c) O tempo máximo na fila
d) O número médio da fila
e) Verifique o tempo ideal de simulação que mostra que os dados não se alteram muito

Incluindo uma parada do recurso

In [ ]:
import simpy
import numpy as np

# Definição dos parâmetros
tempo_chegada = 30  # Média de chegada dos carros (segundos)
tempo_atendimento = 20  # Média de atendimento (segundos)
tempo_folga = 5 * 60  # Tempo de folga do atendente (5 minutos)
tempo_total = 5 * 3600  # Tempo total da simulação (2 horas)


def gerenciar_folga(env):
  global em_folga, folga_pendente
  while True:
    yield env.timeout(3600)  # Espera 1 hora
    if atendente.count > 0:  # Se estiver atendendo, marca a folga como pendente
      print(f"Atendente adiando folga para terminar atendimento em {env.now:.2f}s")
      folga_pendente = True
    else:
      print(f"Atendente iniciando folga em {env.now:.2f}s")
      em_folga = True
      yield env.timeout(tempo_folga)  # Tempo de descanso
      em_folga = False
      print(f"Atendente retornou ao trabalho em {env.now:.2f}s")

def atender_carro(carro_id):
  global em_folga, folga_pendente
  with atendente.request() as req:
    yield req
    while em_folga:
      yield env.timeout(1)  # Aguarda até o fim da folga
    yield env.timeout(np.random.exponential(tempo_atendimento))
    print(f"Carro {carro_id} atendido em {env.now:.2f}s")
    if folga_pendente:  # Se folga estiver pendente, inicia após atendimento
       folga_pendente = False
       print(f"Atendente iniciando folga em {env.now:.2f}s")
       em_folga = True
       yield env.timeout(tempo_folga)  # Tempo de descanso
       em_folga = False
       print(f"Atendente retornou ao trabalho em {env.now:.2f}s")

# Processo de chegada dos carros
def chegada_carros(env):
    carro_id = 0
    while True:
        yield env.timeout(np.random.exponential(tempo_chegada))
        carro_id += 1
        print(f"Carro {carro_id} chegou em {env.now:.2f}s")
        env.process(atender_carro(carro_id))

# Inicialização da simulação
env = simpy.Environment()
atendente = simpy.Resource(env, capacity=1)
folga_pendente = False
em_folga = False
env.process(gerenciar_folga(env))
env.process(chegada_carros(env))
env.run(until=tempo_total)

Exercício 2: Considere uma lavanderia com 4 lavadoras, 3 secadoras e 5 cestos de roupas. Quando um cliente chega (taxa Exponential(3)), ele coloca as roupas em uma máquina de lavar (ou aguarda em fila). A lavagem consome 20 minutos (constante). Ao terminar a lavagem, o cliente retira as roupas da máquina e coloca em um cesto e leva o cesto com suas roupas até a secadora, num processo que leva de 1 a 4 minutos distribuídos uniformemente. O cliente então descarrega as roupas do cesto diretamente para a secadora, espera a secagem e vai embora. Esse processo leva entre 9 e 12 minutos, uniformemente distribuídos. Construa um modelo de simulação que represente este processo. Simule por 10 horas e verifique o gargalo do sistema.